In [ ]:
"""
Decompose Phi-3 into constituent parts. Runs each part peice by piece. Test logit lens, position loss, and feature clustering via the MLP layer.
"""
None

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
from tqdm import tqdm
import numpy as np
import pandas as pd
from IPython.core.display import HTML, Markdown
import plotly.express as px 
import os
import wandb 
from dotenv import load_dotenv
from datetime import datetime
import pytz
from torch.utils.data import DataLoader
import importlib
import pathlib

from transformers import AutoModelForCausalLM, AutoTokenizer
from helpers.phi3.phi3 import Phi3Config, Phi3ForCausalLM, _prepare_4d_causal_attention_mask
from helpers.phi3.parse import parse_phi
from helpers.memory import check_memory

load_dotenv('secrets.env')
device = 'cuda'

## Initialize Model

In [ ]:
attn_implementation = None # None/flash_attention_2

# Load Model
# Padding side not important EXCEPT for flash attention, needs to be left
tokenizer = AutoTokenizer.from_pretrained('microsoft/Phi-3-mini-4k-instruct', add_eos_token = False, add_bos_token = False, padding_side = 'left') 

# Load the usual model from HF transformers - attn_implementation = None to disable flash attention
my_model = AutoModelForCausalLM.from_pretrained(
    'microsoft/Phi-3-mini-4k-instruct',
    device_map = device,
    trust_remote_code = True, 
    torch_dtype = torch.bfloat16, 
    attn_implementation = attn_implementation
    ).to(device).eval()

## Initialize FENCE Parameters

In [ ]:
# Pass indices starting at 1
fence_dict = {
    'dogs': (3064, 3072),
    'test': (3050, 3052)
}

Kfstart = 1
Kfend = 32
Kf_target_values = {
    'hkrs': {Kfstart + j - 1: (j - 1) * .25 + .25/2 for j in range(Kfstart, Kfend + 1)},
    'hks': {Kfstart + j - 1: (j - 1) * .25 + .25 for j in range(Kfstart, Kfend + 1)},
}

print(fence_dict)
Kf_target_values

## Test Inference & Visualizations with Base Model

In [ ]:
importlib.reload(importlib.import_module('helpers.fence.eval'))
from helpers.fence.eval import generate_fence

# Test
dog_prompt = parse_phi([{'role': 'user', 'content': 'What\'s your favorite animal?'}], True)
dog_gens = generate_fence(my_model, tokenizer, prompt = dog_prompt, max_tokens = 12)

animal_prompt = '<s>Animals are,'
animal_gens = generate_fence(my_model, tokenizer, prompt = animal_prompt, max_tokens = 12)

In [ ]:
importlib.reload(importlib.import_module('helpers.fence.visualize'))
from helpers.fence.visualize import visualize_fence, visualize_fence_by_layer, visualize_fence_differentials

for l in [1, 15, 30]:
    visualize_fence(
        dog_gens['text'],
        dog_gens['hks'],
        [l],
        fence_dict,
        start_dim = 2950, end_dim = 3072,
        min_range = 0, max_range = Kf_target_values['hks'][l]
    ).update_layout(title = 'H<sup>R</sup><sub>' + str(l) + '</sub>', height = 300).show('colab')

## Test Component-by-Component Inference

In [ ]:
from helpers.misc import is_notebook
from helpers.phi3.phi3 import _prepare_4d_causal_attention_mask, apply_rotary_pos_emb
import math

@torch.no_grad()
def generate_fence_with_force(model, tokenizer, prompt, echo_output = True, max_tokens = 128, device = 'cuda'):
    """
    Runs a forward pass and stores FENCE-relevant intermediate hidden states. Allows for forced-FENCE (see foward pass code).
    Also calculates the modularity loss. Position loss is NOT calculated as there are no true-FENCE states.

    Returns a dictionary with keys:
        - `text`: The decoded output text, as a list
        - `hk1s`: The first residual stream output
        - `hk2s`: The final residual stream output
        - `hksas`: The hidden state outputs of the SA component
        - `hkmlps`: The hidden state outputs of the MLP component
    """
    model.eval()
    generated_tokens = 0
    
    input_ids_0 = tokenizer(prompt, return_tensors = 'pt').to(device)['input_ids']
    input_ids = input_ids_0

    while True:
        embeds_output = model.model.embed_tokens(input_ids)
        hidden_state = embeds_output
        
        B, N, D = embeds_output.shape
        H = 32
        Dh = int(D/H)
        
        position_ids = torch.arange(0, N, dtype=torch.long, device=device).unsqueeze(0).view(-1, N) # Create position IDs
        
        # Flash attention = use default attention mask 2d
        if model.model._attn_implementation == 'flash_attention_2':
            attention_mask = None
        # Non flash-attention: Make a triangular attention mask to hide right context
        else:
            attention_mask = _prepare_4d_causal_attention_mask(None, (B, N), embeds_output, 0, sliding_window = model.model.config.sliding_window) 

        saved_sa_outputs = []
        saved_hkrs = []
        saved_mlp_outputs = []
        saved_hks = []
        ### Transformer Blocks ###
        for i, layer in enumerate(model.model.layers):            

            residual = hidden_state
            sa_input = layer.input_layernorm(hidden_state)
            
            ### SA ###
            sa_module = layer.self_attn
            # sa_output = sa_module(sa_input, attention_mask, position_ids)[0]
            qkv = sa_module.qkv_proj(sa_input)
            queries = qkv[:, :, :D].view(B, N, H, Dh).transpose(1, 2)
            keys = qkv[:, :, D:2*D].view(B, N, H, Dh).transpose(1, 2)
            values = qkv[:, :, 2*D:].view(B, N, H, Dh).transpose(1, 2)

            if model.model._attn_implementation == 'flash_attention_2':     
                # Flash attention requires the input to have the shape B x N x Dh x D           
                # Because the input can be padded, the absolute sequence length depends on the max position id.
                rotary_seq_len = max(N, position_ids[:, -1].max().item()) + 1
                cos, sin = sa_module.rotary_emb(values, position_ids, seq_len = rotary_seq_len)
                queries, keys = apply_rotary_pos_emb(queries, keys, cos, sin, position_ids)
                ################## # Reshape to the expected shape for Flash Attention
                queries = queries.transpose(1, 2)
                keys = keys.transpose(1, 2)
                values = values.transpose(1, 2)
                ###################
                sa_output = sa_module._flash_attention_forward(queries, keys, values, attention_mask, N)
                sa_output = sa_output.reshape(B, N, D).contiguous()
            else:    
                cos, sin = sa_module.rotary_emb(values, position_ids, seq_len = N)
                queries, keys = apply_rotary_pos_emb(queries, keys, cos, sin, position_ids)
                attn_weights = torch.matmul(queries, keys.transpose(2, 3))/math.sqrt(Dh)  # Should be shape B x H x N x N
                attn_weights = attn_weights + attention_mask # Attemtion mask is upper triangular of negative infinity
                attn_weights = F.softmax(attn_weights, dim = -1, dtype = torch.float32).to(values.dtype)
                sa_output = torch.matmul(attn_weights, values) # B x H x N x D/H
                sa_output = sa_output.transpose(1, 2).contiguous() # Reorder into B x N x H x D/H
                sa_output = sa_output.reshape(B, N, D) # Concatenate vertically back into B x N x D
    
            # Finally post-concatenation linear layer
            sa_output = sa_module.o_proj(sa_output)

            saved_sa_outputs.append(sa_output[0, :, :].detach())
            
            ### add residual -> store residual -> layernorm -> mlp -> add residual
            hidden_state = residual + sa_output

            # FENCE
            if l >= Kfstart - 1 and l <= Kfend - 1:
                # To extract the right layer from hkr_feature_targets:
                # - We want to extract layer l + 1 (e.g. l = 1 => Layer = 2)
                # - Since hkr_feature_targets[:, k, :, :] contains layer Kfstart+k, we want to find k s.t. Kfstart + k = l + 1
                # - => k = l + 1 - Kfstart
                # hidden_state[:, 1:, 3064:3072] = Kf_target_values['hkrs'][l+1] # B x N x Df                
                pass
            
            residual = hidden_state
            saved_hkrs.append(hidden_state[0, :, :].detach())

            hidden_state = layer.post_attention_layernorm(hidden_state)
            ## MLP            
            up_state = layer.mlp.gate_up_proj(hidden_state) # B x N x (2I, I = intermediate MLP dimension)
            gate, up_state = up_state.chunk(2, dim = -1) # B x N x I
            up_state = up_state * layer.mlp.activation_fn(gate)  # Elementwise
            hidden_state = layer.mlp.down_proj(up_state) # Back to B x N x D
            ## End MLP
            
            saved_mlp_outputs.append(hidden_state[0, :, :].detach())

            hidden_state = residual + hidden_state

            if l >= Kfstart - 1 and l <= Kfend - 1:
                # hidden_state[:, 1:, 3064:3072] = Kf_target_values['hks'][l+1] # B x N x Df
                pass

            saved_hks.append(hidden_state[0, :, :].detach())
                
        hidden_state = model.model.norm(hidden_state)
        logits = model.lm_head(hidden_state)

        # Get argmax tokens + concatenate onto previous tokens
        output_token = torch.argmax(F.softmax(logits.squeeze(), dim = 1), dim = 1)[-1]
        input_ids = torch.cat((input_ids, output_token.view(1, 1)), dim = 1)

        # Break while loop if EOS or generation > max tokens
        generated_tokens = generated_tokens + 1
        if output_token in [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|end|>")] or generated_tokens >= max_tokens:
            break

    # Use it on the last pass only
    all_hksas = [h.cpu().to(torch.float16).numpy() for h in saved_sa_outputs]
    all_hkrs = [h.cpu().to(torch.float16).numpy() for h in saved_hkrs]
    all_hkmlps = [h.cpu().to(torch.float16).numpy() for h in saved_mlp_outputs]
    all_hks = [h.cpu().to(torch.float16).numpy() for h in saved_hks]

    final_output = input_ids.squeeze()
    decoded_text = tokenizer.batch_decode(final_output)
    decoded_output = tokenizer.decode(final_output[input_ids_0.size()[1]:])

    if echo_output:
        if is_notebook():
            display(HTML(
                '<div style="padding: 1rem 2rem; background-color:honeydew">' + 
                    '<h4>Modified model output</h4>' + 
                    '<span style="color:green">' + tokenizer.batch_decode(input_ids_0)[0][3:] + '</span> ' + 
                    '<span style="color:red">' + decoded_output + '</span>' +
                '</div>'
            ))
        else:
            print(colored(tokenizer.batch_decode(input_ids_0)[0][3:], 'green'), colored(tokenizer.decode(final_output[input_ids_0.size()[1]:]), 'red'))

generate_fence(my_model, tokenizer, prompt = dog_prompt, max_tokens = 12)

In [153]:
# TEST - SINGLE PASS
model = my_model
prompt = dog_prompt

model.eval()
generated_tokens = 0

input_ids_0 = tokenizer(prompt, return_tensors = 'pt').to(device)['input_ids']
input_ids = input_ids_0

embeds_output = model.model.embed_tokens(input_ids)
hidden_state = embeds_output

B, N, D = embeds_output.shape
H = 32
Dh = int(D/H)

position_ids = torch.arange(0, N, dtype=torch.long, device=device).unsqueeze(0).view(-1, N) # Create position IDs

# Flash attention = use default attention mask 2d
if model.model._attn_implementation == 'flash_attention_2':
    attention_mask = None
# Non flash-attention: Make a triangular attention mask to hide right context
else:
    attention_mask = _prepare_4d_causal_attention_mask(None, (B, N), embeds_output, 0, sliding_window = model.model.config.sliding_window) 

saved_sa_outputs = []
saved_hkrs = []
saved_mlp_outputs = []
saved_hks = []
### Transformer Blocks ###
for i, layer in enumerate(model.model.layers):            

    residual = hidden_state
    sa_input = layer.input_layernorm(hidden_state)
    
    ### SA ###
    sa_module = layer.self_attn
    # sa_output = sa_module(sa_input, attention_mask, position_ids)[0]
    qkv = sa_module.qkv_proj(sa_input)
    queries = qkv[:, :, :D].view(B, N, H, Dh).transpose(1, 2)
    keys = qkv[:, :, D:2*D].view(B, N, H, Dh).transpose(1, 2)
    values = qkv[:, :, 2*D:].view(B, N, H, Dh).transpose(1, 2)

    if model.model._attn_implementation == 'flash_attention_2':     
        # Flash attention requires the input to have the shape B x N x Dh x D           
        # Because the input can be padded, the absolute sequence length depends on the max position id.
        rotary_seq_len = max(N, position_ids[:, -1].max().item()) + 1
        cos, sin = sa_module.rotary_emb(values, position_ids, seq_len = rotary_seq_len)
        queries, keys = apply_rotary_pos_emb(queries, keys, cos, sin, position_ids)
        ################## # Reshape to the expected shape for Flash Attention
        queries = queries.transpose(1, 2)
        keys = keys.transpose(1, 2)
        values = values.transpose(1, 2)
        ###################
        sa_output = sa_module._flash_attention_forward(queries, keys, values, attention_mask, N)
        sa_output = sa_output.reshape(B, N, D).contiguous()
    else:    
        cos, sin = sa_module.rotary_emb(values, position_ids, seq_len = N)
        queries, keys = apply_rotary_pos_emb(queries, keys, cos, sin, position_ids)
        attn_weights = torch.matmul(queries, keys.transpose(2, 3))/math.sqrt(Dh)  # Should be shape B x H x N x N
        attn_weights = attn_weights + attention_mask # Attemtion mask is upper triangular of negative infinity
        attn_weights = F.softmax(attn_weights, dim = -1, dtype = torch.float32).to(values.dtype)
        sa_output = torch.matmul(attn_weights, values) # B x H x N x D/H
        sa_output = sa_output.transpose(1, 2).contiguous() # Reorder into B x N x H x D/H
        sa_output = sa_output.reshape(B, N, D) # Concatenate vertically back into B x N x D

    # Finally post-concatenation linear layer
    sa_output = sa_module.o_proj(sa_output)

    saved_sa_outputs.append(sa_output[0, :, :].detach())
    
    ### add residual -> store residual -> layernorm -> mlp -> add residual
    hidden_state = residual + sa_output

    # FENCE
    if l >= Kfstart - 1 and l <= Kfend - 1:
        # To extract the right layer from hkr_feature_targets:
        # - We want to extract layer l + 1 (e.g. l = 1 => Layer = 2)
        # - Since hkr_feature_targets[:, k, :, :] contains layer Kfstart+k, we want to find k s.t. Kfstart + k = l + 1
        # - => k = l + 1 - Kfstart
        # hidden_state[:, 1:, 3064:3072] = Kf_target_values['hkrs'][l+1] # B x N x Df                
        pass
    
    residual = hidden_state
    saved_hkrs.append(hidden_state[0, :, :].detach())

    hidden_state = layer.post_attention_layernorm(hidden_state)
    hidden_state_pre_mlp = hidden_state
    ## MLP       
    # hidden_state is of size B x N x D
    gate_plus_fvals = layer.mlp.gate_up_proj(hidden_state) # B x N x (2I, I = intermediate MLP dimension)
    gate, fvals = gate_plus_fvals.chunk(2, dim = -1) # B x N x I
    # At this point the up_state = values (see Geva et al), and the gate is the keys
    up_state = fvals * layer.mlp.activation_fn(gate)  # Elementwise
    hidden_state = layer.mlp.down_proj(up_state) # Back to B x N x D
    ## End MLP
    
    saved_mlp_outputs.append(hidden_state[0, :, :].detach())

    hidden_state = residual + hidden_state

    if l >= Kfstart - 1 and l <= Kfend - 1:
        # hidden_state[:, 1:, 3064:3072] = Kf_target_values['hks'][l+1] # B x N x Df
        pass

    saved_hks.append(hidden_state[0, :, :].detach())
        
hidden_state = model.model.norm(hidden_state)
logits = model.lm_head(hidden_state)

# Get argmax tokens + concatenate onto previous tokens
output_token = torch.argmax(F.softmax(logits.squeeze(), dim = 1), dim = 1)[-1]
input_ids = torch.cat((input_ids, output_token.view(1, 1)), dim = 1)


tokenizer.batch_decode([output_token])

['As']

In [152]:
fvals

tensor([[[ 0.1992, -0.1187, -0.5352,  ..., -0.2754, -1.1641,  0.6445],
         [-0.1816,  1.8359, -0.9102,  ...,  0.3574, -1.8594,  1.2656],
         [-0.1953,  2.0156,  0.2422,  ...,  1.1172, -0.3496,  0.7734],
         ...,
         [ 0.1133,  1.1875, -0.6523,  ...,  0.1035, -1.4922,  1.1797],
         [-1.1797,  1.8906, -2.5156,  ...,  0.8359, -1.4609, -1.4844],
         [-1.1250,  2.5312, -2.1094,  ...,  0.2217, -1.1172, -0.2129]]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<SplitBackward0>)

In [162]:
def get_modularity_loss(hidden_state, fvals):
    
    B, N, D = hidden_state.shape  # Batch size, Sequence length, Feature dimension
    _, _, I = fvals.shape  # Intermediate dimension

    # Element-wise products in the interaction space (sequence-wise)
    # We need to compute: abs(h_i * h_k * v_ij * v_kj) but only for sequence positions, not their feature products directly

    # First, compute outer products of sequence elements' activations in V space (N x N interactions for each batch):
    # B x N x 1 x I
    fvals_exp = fvals.unsqueeze(2)  
    # B x 1 x N x I
    fvals_exp_k = fvals.unsqueeze(1)

    # Calculate the element-wise absolute products
    interaction_terms = torch.abs(fvals_exp * fvals_exp_k)  # B x N x N x I

    # Now, sum across the interaction dimension I for each pair (i, k):
    interaction_sums = interaction_terms.sum(dim=-1)  # B x N x N

    # Calculate |i - k| using broadcasting
    i_indices = torch.arange(N).view(1, -1, 1)  # 1 x N x 1
    k_indices = torch.arange(N).view(1, 1, -1)  # 1 x 1 x N
    distances = torch.abs(i_indices - k_indices)  # N x N
    distances = distances.to(hidden_state.device)  # Ensure distances are on the same device

    # Broadcast distances to match interaction terms
    distances_exp = distances.unsqueeze(0)  # 1 x N x N

    # Calculate the raw L1 norm loss: multiply the interactions by their distances
    L1_raw = (interaction_sums * distances_exp).sum(dim=(1, 2))  # Sum over N x N

    # Calculate the sum of interactions for normalization
    sum_interactions = interaction_sums.sum(dim=(1, 2))  # Sum over N x N

    # Normalize the loss, handling division by zero
    L1_norm_loss = torch.where(sum_interactions != 0, L1_raw / sum_interactions, torch.tensor(0.0).to(hidden_state.device))

    # Average loss across the batch
    return L1_norm_loss.mean()

get_modularity_loss(hidden_state_pre_mlp, fvals)

tensor(0.3145, device='cuda:0', dtype=torch.bfloat16, grad_fn=<MeanBackward0>)

In [157]:
hidden_state_pre_mlp.shape

torch.Size([1, 11, 3072])